# bm25 llamaindex 支持中文

## 准备工作

In [10]:
%%time
%%capture

%pip install rank_bm25
%pip install nltk jieba

!pip install llama-index-retrievers-bm25==0.1.3
!pip install rank_bm25
!pip install jieba

CPU times: user 27.2 ms, sys: 52.4 ms, total: 79.6 ms
Wall time: 11 s


In [3]:
%%time

# 下载停用词

# 设置 HTTP 代理环境变量
# https://github.com/nltk/nltk_data/issues/154#issuecomment-2144880495
http_proxy="http://192.168.0.134:7890"

import nltk
nltk.set_proxy(f'{http_proxy}')
nltk.download('stopwords')

CPU times: user 1.23 s, sys: 304 ms, total: 1.53 s
Wall time: 1.38 s


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
%%time
import jieba
from typing import List
from nltk.corpus import stopwords

def chinese_tokenizer(text: str) -> List[str]:
    # Use jieba to segment Chinese text
    # return list(jieba.cut(text))
    # tokens = jieba.lcut(text)
    # return [token for token in tokens if token not in stopwords.words('chinese')]
    # print(list(jieba.lcut(text)))
    return list(jieba.lcut(text))

CPU times: user 28 μs, sys: 0 ns, total: 28 μs
Wall time: 30 μs


In [5]:
chinese_tokenizer("床前明月光")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.600 seconds.
Prefix dict has been built successfully.


['床前', '明月光']


['床前', '明月光']

In [6]:
%%time

from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.response.notebook_utils import display_source_node

documents = [Document(text="床前明月光"),
             Document(text="疑是地上霜"),
             Document(text="举头望明月"),
             Document(text="低头思故乡")]

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

nodes

CPU times: user 1.74 s, sys: 146 ms, total: 1.88 s
Wall time: 2.09 s


[TextNode(id_='e477a0c9-2f42-45e8-a088-d8ca2a7c73a9', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1dab2bc5-998e-4fc6-b848-bdc9a0c5b12e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='ac0f389f8d076d84f369d702eaf943bb5a090d8aa88d9db41a0f79cbc098ea86')}, text='床前明月光', mimetype='text/plain', start_char_idx=0, end_char_idx=5, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='bfc8b01b-be84-49bd-a865-46e93d423eac', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0e391e41-6548-4f60-9c79-d9d40def764b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='761a48a5e3191f70fdb9c7799ef3c6168a3cc0db449cb4a7b40b0b450fd66fd9')}, text='疑是地上霜', mimetype='text/plain', start_char_idx=0,

## 使用 retriver

In [7]:
%%time

retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=1,
    tokenizer=chinese_tokenizer,
)

['床前', '明月光']
['疑是', '地上', '霜']
['举头', '望明月']
['低头', '思', '故乡']
CPU times: user 731 μs, sys: 118 μs, total: 849 μs
Wall time: 846 μs


In [8]:
retriever.retrieve("故乡")

['故乡']


[NodeWithScore(node=TextNode(id_='cf945d4c-6945-405c-89f2-7fa46b67b9e8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0b944634-f091-4705-b894-e7ab4da03a24', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='d5b6840ed7517b443f5a2bf10b5aa511799e6715c432857123f40212514233b2')}, text='低头思故乡', mimetype='text/plain', start_char_idx=0, end_char_idx=5, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.777337486593765)]

In [9]:
retriever.retrieve("举头")

['举头']


[NodeWithScore(node=TextNode(id_='6e67b50e-11d5-4b2e-866e-c143eadc9d4c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='746c5193-fc6c-42b0-84ee-e3f1661f8a78', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='42bf7864167ad4408fb5929d538790398e1be8b5c36dffb328dc493f94cd0f64')}, text='举头望明月', mimetype='text/plain', start_char_idx=0, end_char_idx=5, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.9310965498760481)]

## 更复杂的示例

### 创建新闻摘要文档

In [11]:
%%time

documents = [Document(text="欧盟委员会首席发言人埃里克·马梅宣布，由于匈牙利担任欧盟理事会轮值主席国以来的事态发展，欧盟委员会主席冯德莱恩决定只派高级公务员代表参加理事会非正式会议，而不是派遣高级官员。匈牙利方面对此表示质疑，认为欧盟委员会不能挑选它想要合作的机构和成员国。"),
             Document(text="文章由专栏作者青岚撰写，主要探讨了荷兰光刻机巨头ASML公司在四十年的芯片产业发展中的变迁，以及它如何从最初的市场挑战者变成了当前全球半导体产业政治博弈中的焦点。"),
             Document(text="文章由华阳海洋研究中心理事长吴士存撰写，讨论了近期中菲在南海仁爱礁的对峙问题，并提出了中国应对该问题的建议。"),
             Document(text="报道主要关注了孟加拉国总理哈西娜在提斯塔河项目上的立场，以及孟加拉国和印度在水资源分配问题上的长期争端。哈西娜倾向于让印度负责项目，以确保孟加拉国的水资源需求得到满足。")]

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

CPU times: user 4.83 ms, sys: 132 μs, total: 4.96 ms
Wall time: 3.95 ms


### 使用 BM25Retriever

In [13]:
%%time

retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=1,
    tokenizer=chinese_tokenizer,
)

CPU times: user 2.1 ms, sys: 57 μs, total: 2.16 ms
Wall time: 2.16 ms


In [15]:
%%time

retriever.retrieve("欧盟匈牙利轮值主席国后发生了啥")

CPU times: user 1.54 ms, sys: 270 μs, total: 1.81 ms
Wall time: 1.59 ms


[NodeWithScore(node=TextNode(id_='7ffbb516-c680-4b4b-9a8d-d21e94ce53d8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='26aaefef-1605-4b60-9483-75591c125d5b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='da0f80d48d2c7374c50a4b3ef22dce19c723c2e43db9a6c6110637d697f5466a')}, text='欧盟委员会首席发言人埃里克·马梅宣布，由于匈牙利担任欧盟理事会轮值主席国以来的事态发展，欧盟委员会主席冯德莱恩决定只派高级公务员代表参加理事会非正式会议，而不是派遣高级官员。匈牙利方面对此表示质疑，认为欧盟委员会不能挑选它想要合作的机构和成员国。', mimetype='text/plain', start_char_idx=0, end_char_idx=123, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=3.375648286021731)]

In [16]:
%%time

retriever.retrieve("荷兰asml公司发展有哪些故事")

CPU times: user 1.77 ms, sys: 0 ns, total: 1.77 ms
Wall time: 1.5 ms


[NodeWithScore(node=TextNode(id_='9bd045ab-4761-4b70-aab9-bfde29da7cd1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='849e5b40-4d6a-452a-ac8e-d39153fc8a87', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='678927f4d186eb02e8f1ca9fd066c9c9fa333e2a8d25bf50904d2de9f094c987')}, text='文章由专栏作者青岚撰写，主要探讨了荷兰光刻机巨头ASML公司在四十年的芯片产业发展中的变迁，以及它如何从最初的市场挑战者变成了当前全球半导体产业政治博弈中的焦点。', mimetype='text/plain', start_char_idx=0, end_char_idx=81, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1.6903462469691775)]

In [17]:
%%time

retriever.retrieve("仁爱礁下一步会发生什么事情")

CPU times: user 1.83 ms, sys: 19 μs, total: 1.85 ms
Wall time: 1.6 ms


[NodeWithScore(node=TextNode(id_='9b8d6593-8bd0-4e5a-9c30-eb68de901e98', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='fa8de683-6c5b-40b0-a768-20c91f86f429', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='e7d871f0cb2a5b3fca85ef67599dec9c170018551a96b3b29db2589bdc49c7df')}, text='文章由华阳海洋研究中心理事长吴士存撰写，讨论了近期中菲在南海仁爱礁的对峙问题，并提出了中国应对该问题的建议。', mimetype='text/plain', start_char_idx=0, end_char_idx=54, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1.9438169434067216)]